In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
from tensorflow.keras.preprocessing.image import load_img,img_to_array
import os
import matplotlib.pyplot as plt
import numpy as np
'''
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        paths.append(os.path.join(dirname, filename))
'''
path='/kaggle/input/images-for-style-transfer/Data/Artworks'
styles=list()
for file in os.listdir(path):
    img=load_img(os.path.join(path,file),target_size=(128,128,3))
    styles.append(img_to_array(img))
styles=np.asarray(styles)/255
plt.imshow(styles[1])
# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import math,datetime
import numpy as np
import pandas as pd
from sklearn import preprocessing,svm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from matplotlib import style
import pickle
import quandl

style.use('ggplot')
quandl.ApiConfig.api_key='4YW-QTzBhE_R2tmcVyR6'
df=quandl.get('WIKI/GOOGL')

In [ ]:
from sklearn.decomposition import PCA
col=df.columns
X=np.array(df)
X=preprocessing.scale(X)
pca=PCA()
pca.fit(X)
cov=pca.get_covariance()
np.fill_diagonal(cov,0.0)
co=cov.astype('int32')
co=np.where(co==1)
x_co,y_co=co[0],co[1]
for x,y in zip(x_co,y_co):
    print(col[x],col[y],cov[x][y],sep=',')

In [ ]:
df=df[['Adj. High','Adj. Low','Adj. Open','Adj. Close','Volume']]
df['HL_PCT']=(df['Adj. High']-df['Adj. Low'])*100.0/df['Adj. Low']
df['PCT_Change']=(df['Adj. Close']-df['Adj. Open'])*100.0/df['Adj. Open']
df=df[['Adj. Close','HL_PCT','PCT_Change','Volume']]

In [ ]:
forecast_col='Adj. Close'
forecast_out=int(math.ceil(0.01*len(df)))
df['Label']=df[forecast_col].shift(-forecast_out)
'''
X_lately=np.array(df[-forecast_out:])
df.dropna(axis=0,inplace=True)
X=np.array(df.drop('Label',axis=1))
y=np.array(df['Label'])
''' 
X=np.array(df.drop('Label',axis=1))
X=preprocessing.scale(X)
X_lately=X[-forecast_out:]
X=X[:-forecast_out]
df.dropna(axis=0,inplace=True)
y=np.array(df['Label'])

In [ ]:
#X=preprocessing.scale(X)
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)
#f=open('LinearRegression.pickle','rb')
clf=LinearRegression(n_jobs=-1)
clf.fit(X_train,y_train)
acc_=clf.score(X_test,y_test)
print(acc_)

In [ ]:
forecast_set=clf.predict(X_lately)
plt.plot(df['Label'][:35])
last_date=df.iloc[-1].name
last_unix=last_date.timestamp()
one_day=86400
next_unix=last_unix+one_day
for i in forecast_set:
    next_day=datetime.datetime.fromtimestamp(next_unix)
    next_unix+=one_day
    df.loc[next_day]=[np.nan for _ in range(len(df.columns)-1)]+[i]
plt.plot(df['Label'][:25])
plt.show()

In [ ]:
import keras
from keras.models import Model
from keras.layers import Dense,Input,Activation
from keras.metrics import categorical_crossentropy
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import style
import pickle
style.use('ggplot')
X=np.arange(1,7)
y=np.array([5,4,6,5,6,7])
input_param=Input(shape=(1,))
hidden_layer=Dense(16,activation='linear')(input_param)
output_layer=Dense(1,activation='linear')(hidden_layer)
clf=Model(input_param,output_layer)
loss_fn=keras.losses.MeanSquaredError()
clf.compile(optimizer='Adam',loss=loss_fn)
clf.fit(X,y,verbose=2,epochs=2000,batch_size=1,shuffle=True)

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input,Conv2D,MaxPooling2D,Conv2DTranspose,BatchNormalization,LeakyReLU
from tensorflow.keras.preprocessing.image import load_img,img_to_array
import os
import numpy as np
import matplotlib.pyplot as plt
def get_data(filename,data):
    for file in os.listdir(filename):
        img=load_img(os.path.join(filename,file),target_size=(128,128,3))
        img=img_to_array(img)
        data.append(img)
    return data
dataset=list()
path='/kaggle/input/natural-images/natural_images'
for file_dir in os.listdir(path):
    dataset=get_data(os.path.join(path,file_dir),dataset)
dataset=np.asarray(dataset,dtype='float32')
dataset=dataset/255
dataset.shape

In [ ]:
from tensorflow.keras.layers import Add,Activation
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow as tf
def resnet_block(n_filters,ip_model):
    d=Conv2D(n_filters,(3,3),padding='same')(ip_model)
    d=BatchNormalization()(d)
    d=Activation('relu')(d)
    
    d=Conv2D(n_filters,(3,3),padding='same')(d)
    d=BatchNormalization()(d)
    d=Activation('relu')(d)
    
    d=Add()([d,ip_model])
    return d
def load_mod():
    path='/kaggle/input/aemodel/ae.h5'
    ae=get_model(dataset[1].shape,8)
    ae.load_weights(path)
    
    de=tf.keras.models.Sequential()
    for layer in ae.layers[70:]:
        de.add(layer)
    de.build((1,64,64,256))
    
    en=Model(ae.input,ae.layers[69].output)
    return ae,en,de

def get_model(ip_shape,n_resnets):
    ip=Input(shape=ip_shape)
    '''
    d=Conv2D(64,(3,3),padding='same')(ip)
    d=BatchNormalization()(d)
    d=Activation('relu')(d)
    '''
    d=Conv2D(128,(3,3),strides=2,padding='same')(ip)
    d=BatchNormalization()(d)
    d=Activation('relu')(d)
    
    d=Conv2D(256,(3,3),strides=2,padding='same')(d)
    d=BatchNormalization()(d)
    d=Activation('relu')(d)
    '''
    d=Conv2D(256,(3,3),padding='same')(d)
    d=BatchNormalization()(d)
    d=Activation('relu')(d)
    '''
    for i in range(n_resnets):
        d=resnet_block(256,d)
        
    encoded=resnet_block(256,d)
    
    d=Conv2DTranspose(128,(3,3),strides=2,padding='same')(encoded)
    d=BatchNormalization()(d)
    d=Activation('relu')(d)
    
    d=Conv2DTranspose(64,(3,3),strides=2,padding='same')(d)
    d=BatchNormalization()(d)
    d=Activation('relu')(d)
    '''
    d=Conv2DTranspose(64,(3,3),strides=2,padding='same')(d)
    d=BatchNormalization()(d)
    d=Activation('relu')(d)
    '''
    d=Conv2DTranspose(3,(3,3),padding='same')(d)
    d=BatchNormalization()(d)
    decoded=Activation('relu')(d)
    
    #e=Input(shape=(64,64,256))
    autoencoder=Model(ip,decoded)
    return autoencoder

ae,encoder,decoder=load_mod()

def ssim_loss(y_true,y_pred):
    return tf.reduce_mean(tf.image.ssim(y_true,y_pred,1.0))

cp=ModelCheckpoint(filepath='/kaggle/working/ae',save_best_only=True,monitor='val_acc',mode=' max',save_weights_only=True)
ae.compile(optimizer='adam',loss='mse',metrics=['accuracy'])
#ae.fit(dataset,dataset,verbose=2,epochs=100,batch_size=64,shuffle=True,validation_split=0.2,callbacks=[cp])
#ae.summary()

In [ ]:
n_samples=5
sample_data=dataset[np.random.randint(0,6888,5)]
predictions=np.asarray(ae.predict(sample_data))
for i in range(n_samples):
    plt.subplot(2,n_samples,i+1)
    plt.imshow(sample_data[i])
for i in range(n_samples):
    plt.subplot(2,n_samples,i+6)
    plt.imshow(predictions[i])
plt.show()

In [ ]:
ae.save_weights('/kaggle/working/ae.h5')

In [ ]:
data1=styles[0]
plt.subplot(1,3,1)
plt.imshow(data1)

data2=styles[1]
plt.subplot(1,3,2)
plt.imshow(data2)

data1_en=encoder.predict(tf.expand_dims(data1,axis=0))
data2_en=encoder.predict(tf.expand_dims(data2,axis=0))
data_dec=decoder.predict(np.add(data1_en,data2_en*0.8))[0]
plt.subplot(1,3,3)
plt.imshow(data_dec)